# 데이터 분할

이전 [노트북](https://github.com/rickiepark/ml-powered-applications/blob/master/notebooks/dataset_exploration.ipynb)에서 데이터셋을 탐색했습니다. 이제 이 데이터셋을 훈련 세트와 테스트 세트로 분할하겠습니다. 데이터셋을 분할하는 것은 모델의 성능을 검증하는데 매우 중요합니다. 일부 데이터로만 모델을 훈련하고 모델의 실전 성능이 얼마나 될지 추정하기 위해 본 적 없는 데이터를 사용할 수 있습니다.

이 노트북에서 `writer` 스택 오버플로 데이터셋을 사용해 몇 가지 분할 방법을 소개합니다. 먼저 데이터를 로드하고 전처리합니다.

In [1]:
def format_raw_df(df):
    """
    데이터를 정제하고 질문과 대답을 합칩니다.
    """
    df["PostTypeId"] = df["PostTypeId"].astype(int)
    df["Id"] = df["Id"].astype(int)
    df["AnswerCount"] = df["AnswerCount"].fillna(-1)
    df["AnswerCount"] = df["AnswerCount"].astype(int)
    df["OwnerUserId"].fillna(-1, inplace=True)
    df["OwnerUserId"] = df["OwnerUserId"].astype(int)
    df.set_index("Id", inplace=True, drop=False)

    df["is_question"] = df["PostTypeId"] == 1

    # 문서화된 것 이외의 PostTypeId를 필터링한다
    df = df[df["PostTypeId"].isin([1, 2])]

    # 질문과 대답을 연결합니다
    df = df.join(
        df[["Id", "Title", "body_text", "Score", "AcceptedAnswerId"]],
        on="ParentId",
        how="left",
        rsuffix="_question",
    )
    return df

In [2]:
from sklearn.model_selection import train_test_split

def get_random_train_test_split(posts, test_size=0.3, random_state=40):
    """
    DataFrame을 훈련/테스트 세트로 나눕니다.
    DataFrame이 질문마다 하나의 행을 가진다고 가정합니다.
    :param posts: 모든 포스트와 레이블
    :param test_size: 테스트 세트로 할당할 비율
    :param random_state: 랜덤 시드
    """
    return train_test_split(
        posts, test_size=test_size, random_state=random_state
    )

In [3]:
def get_vectorized_inputs_and_label(df):
    """
    DataFrame 특성과 텍스트 벡터를 연결합니다.
    :param df: 계산된 특성의 DataFrame
    :return: 특성과 텍스트로 구성된 벡터
    """
    vectorized_features = np.append(
        np.vstack(df["vectors"]),
        df[
            [
                "action_verb_full",
                "question_mark_full",
                "norm_text_len",
                "language_question",
            ]
        ],
        1,
    )
    label = df["Score"] > df["Score"].median()

    return vectorized_features, label

In [4]:
from sklearn.model_selection import GroupShuffleSplit

def get_split_by_author(
    posts, author_id_column="OwnerUserId", test_size=0.3, random_state=40
):
    """
    훈련 세트와 테스트 세트로 나눕니다.
    작성자가 두 세트 중에 하나에만 등장하는 것을 보장합니다.
    :param posts: 모든 포스트와 레이블
    :param author_id_column: author_id가 들어 있는 열 이름
    :param test_size: 테스트 세트로 할당할 비율
    :param random_state: 랜덤 시드
    """
    splitter = GroupShuffleSplit(
        n_splits=1, test_size=test_size, random_state=random_state
    )
    splits = splitter.split(posts, groups=posts[author_id_column])
    train_idx, test_idx = next(splits)
    return posts.iloc[train_idx, :], posts.iloc[test_idx, :]


In [6]:
import pandas as pd
import spacy
import umap
import numpy as np
from pathlib import Path
import sys
sys.path.append("..")
import warnings
warnings.filterwarnings('ignore')

data_path = Path('data/writers.csv')
df = pd.read_csv(data_path)
df = format_raw_df(df.copy())

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33650 entries, 1 to 42885
Data columns (total 29 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 33650 non-null  int64  
 1   AcceptedAnswerId           4124 non-null   float64
 2   AnswerCount                33650 non-null  int64  
 3   Body                       33650 non-null  object 
 4   ClosedDate                 969 non-null    object 
 5   CommentCount               33650 non-null  int64  
 6   CommunityOwnedDate         186 non-null    object 
 7   CreationDate               33650 non-null  object 
 8   FavoriteCount              3307 non-null   float64
 9   Id                         33650 non-null  int64  
 10  LastActivityDate           33650 non-null  object 
 11  LastEditDate               10521 non-null  object 
 12  LastEditorDisplayName      606 non-null    object 
 13  LastEditorUserId           9975 non-null   float64


## 랜덤 분할

테스트 세트를 만드는 가장 간단한 방법은 랜덤하게 데이터를 훈련 세트와 테스트 세트로 나누는 것입니다. 방식은 다음과 같습니다.

In [9]:
train_df_rand, test_df_rand = get_random_train_test_split(df[df["is_question"]], test_size=0.3, random_state=40)

In [11]:
print("훈련 세트: %s개 질문, 테스트 세트: %s개 질문" % (len(train_df_rand), len(test_df_rand)))
train_owners = set(train_df_rand['OwnerUserId'].values)
test_owners = set(test_df_rand['OwnerUserId'].values)
print("훈련 세트에 있는 작성자: %s명" % len(train_owners))
print("테스트 세트에 있는 작성자: %s명" % len(test_owners))
print("양쪽에 모두 등장하는 작성자: %s명" % len(train_owners.intersection(test_owners)))

훈련 세트: 5579개 질문, 테스트 세트: 2392개 질문
훈련 세트에 있는 작성자: 2955명
테스트 세트에 있는 작성자: 1531명
양쪽에 모두 등장하는 작성자: 596명


이 방식은 한 가지 단점이 있습니다. 다음 섹션으로 넘어가기 전에 이 단점이 무엇인지 생각해 보세요.

## 작성자를 기준으로 분할하기

어떤 작성자는 다른 사람보다 질문을 작성하는데 더 뛰어날 수 있습니다. 한 작성자가 훈련 세트와 테스트 세트에 모두 등장하면 모델이 작성자를 인식하여 간단히 질문의 점수를 예측할 수 있습니다. 단순하게 특성에서 `AuthorId`를 삭제하는 것은 이 문제를 완전히 해결하지 못합니다. 질문에 저자의 특징이 포함되어 있을 수 있기 때문입니다(특히 일부 작성자는 자기 사인을 질문에 포함시킵니다).

질문의 품질을 정확하게 판단하기 위해서 한 작성자는 훈련 세트나 검증 세트 하나에만 등장해야 합니다. 이를 통해 모델이 저자를 식별할 수 있는 정보를 사용하여 쉽게 예측을 하지 못하게 만들 수 있습니다.

이런 잠재적인 편향의 원인을 제거하기 위해 작성자를 기준으로 분할하겠습니다.

In [12]:
train_author, test_author = get_split_by_author(df[df["is_question"]], test_size=0.3, random_state=40)

print("훈련 세트: %s개 질문, 테스트 세트: %s개 질문" % (len(train_author),len(test_author)))
train_owners = set(train_author['OwnerUserId'].values)
test_owners = set(test_author['OwnerUserId'].values)
print("훈련 세트에 있는 작성자: %s명" % len(train_owners))
print("훈련 세트에 있는 작성자: %s명" % len(test_owners))
print("양쪽에 모두 등장하는 작성자: %s명" % len(train_owners.intersection(test_owners)))

훈련 세트: 5676개 질문, 테스트 세트: 2295개 질문
훈련 세트에 있는 작성자: 2723명
훈련 세트에 있는 작성자: 1167명
양쪽에 모두 등장하는 작성자: 0명


여기서는 작성자를 기준으로 분할하지만 다른 유형의 데이터를 위한 분할 방법이 여러 가지가 있습니다. 예를 들어 어떤 기간 동안 쓰여진 질문에서 훈련하면 최근에 질문에 잘 동작하는 모델을 만들 수 있는지 확인하기 위해 시간 기분으로 분할할 수 있습니다. 더 자세한 내용은 책을 참고하세요.